In [1]:
import sqlite3
from sqlite3 import Error


def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

    return conn

def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
def create_trade_idea(conn, idea):
    """
    Create a new project into the projects table
    :param conn:
    :param project:
    :return: project id
    """
    sql = ''' INSERT OR IGNORE INTO trade_ideas(symbol,time_frame,position,time_posted,user)
              VALUES(?,?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, idea)
    conn.commit()
    return cur.lastrowid

def create_new_table():
    
    database = r"TradingBotDatabase.db"


    # create a database connection
    conn = create_connection(database)
    sql_create_trade_ideas_table = """ CREATE TABLE IF NOT EXISTS stock_data (
                                        symbol text NOT NULL,
                                        timestamp timestamp NOT NULL,
                                        open float NOT NULL,
                                        high float NOT NULL,
                                        low float NOT NULL,
                                        close float NOT NULL,
                                        volumne integer NOT NULL,
                                        PRIMARY KEY(symbol)
                                    ); """
    # create tables
    if conn is not None:
        # create projects table
        create_table(conn, sql_create_trade_ideas_table)
    else:
        print("Error! cannot create the database connection.")

def add_trade_ideas(trade_ideas):
    database = r"TradingBotDatabase.db"


    # create a database connection
    conn = create_connection(database)

    # create tables
#     if conn is not None:
#         # create projects table
#         create_table(conn, sql_create_trade_ideas_table)
#     else:
#         print("Error! cannot create the database connection.")

    with conn:
        # tasks
        for i in trade_ideas:
            print(i)
            trade_idea = (i.symbol, i.time_frame, i.position, i.time_posted, i.user)
            create_trade_idea(conn, trade_idea)

        # create tasks
        
def add_stock_data(stock, symbol):
    database = r"TradingBotDatabase.db"


    # create a database connection
    conn = create_connection(database)

    with conn:
        # tasks
        for i in stock:
            print(i)
            trade_idea = (i.symbol, i.time_frame, i.position, i.time_posted, i.user)
            create_trade_idea(conn, trade_idea)

        # create tasks


In [2]:
import finnhub
import pandas as pd
import datetime
import requests
from bs4 import BeautifulSoup
import re
import time

class trade_idea:
    def __init__(self, symbol, time_frame, position, time_posted, user):
        self.symbol = symbol
        self.time_frame = time_frame
        self.position = position
        self.time_posted = time_posted
        self.user = user
        
def get_trade_ideas(begin_page, end_page):
    trade_ideas = []
    for i in range(begin_page, end_page):
        trade_ideas.extend(store_all_data("https://www.tradingview.com/markets/stocks-usa/ideas/page-%d/" % i))
    print(len(trade_ideas))
    return trade_ideas

def store_all_data(URL):
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")
    data = soup.find_all("div", class_="tv-widget-idea js-userlink-popup-anchor")
    trade_ideas = []
    print(len(data))
    for i in data:
        symbol = i.find('a', class_="tv-widget-idea__symbol apply-overflow-tooltip")
        time_frame = i.find_all('span', class_="tv-widget-idea__timeframe")[1]
        position = i.find('span', class_="content-yHuWj4ze badge-idea-content-Naj79Gc7")
        time_posted = i.find('span', class_="tv-card-stats__time")
        user = i.find('span', class_="tv-card-user-info__name")
        if(symbol != None and time_frame != None and position != None and time_posted != None and user != None):
            trade_ideas.append(trade_idea(symbol.text, time_frame.text, position.text, time_posted['data-timestamp'], user.text))          
    return trade_ideas

def print_trade_ideas():
    for i in trade_ideas:
        print(i.symbol, i.time_frame, i.position, i.time_posted, i.user)

C:\Users\jordi\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\jordi\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\jordi\anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [ ]:
trade_ideas = get_trade_ideas(148,200)

In [51]:
import sqlite3

# Create a SQL connection to our SQLite database
con = sqlite3.connect("TradingBotDatabase.db")

cur = con.cursor()

# The result of a "cursor.execute" can be iterated over by row
# for row in cur.execute('SELECT * FROM trade_ideas ORDER BY time_posted;'):
#     print(row)

collect_symbols = []

for row in cur.execute('SELECT symbol FROM trade_ideas GROUP BY symbol ORDER BY COUNT(symbol) DESC LIMIT 20;'):
    collect_symbols.append(row[0])
    print(row[0])

# for row in cur.execute('SELECT DISTINCT symbol FROM trade_ideas;'):
#     print(row[0])

# Be sure to close the connection
con.close()

SPY
TSLA
AAPL
AMZN
MSFT
QQQ
NVDA
AMD
NFLX
BABA
AMC
META
RBLX
CRWD
GOOG
SMH
GDX
IWM
OXY
PYPL


In [8]:
import alpaca_trade_api as api

API_KEY = 'PKQA6MM20L2CQZKAOAHW'
API_SECRET = '4AJeSo8hcSqHa2TNwJv9IS8efb5PNDxHUju3UJjH'
BASE_URL = "https://paper-api.alpaca.markets"

alpaca = api.REST(API_KEY, API_SECRET, BASE_URL)

In [6]:
def get_crypto_historic_data(symbol, timeframe = "1Min", start = "2022-01-25", end = "2022-01-31", exchange = "CBSE"):
    return alpaca.get_crypto_bars(symbol, timeframe, start, end, exchanges = exchange).df    

def get_stock_historic_data(symbol, timeframe = "1Min", start = "2022-01-25", end = "2022-01-31"):
    return alpaca.get_bars(symbol, timeframe, start, end).df   

In [3]:
collect_symbols = """SPY
TSLA
AAPL
AMZN
MSFT
QQQ
NVDA
AMD
NFLX
BABA
AMC
META
RBLX
CRWD
GOOG
SMH
GDX
IWM
OXY
PYPL""".split()

In [4]:
collect_symbols

['SPY',
 'TSLA',
 'AAPL',
 'AMZN',
 'MSFT',
 'QQQ',
 'NVDA',
 'AMD',
 'NFLX',
 'BABA',
 'AMC',
 'META',
 'RBLX',
 'CRWD',
 'GOOG',
 'SMH',
 'GDX',
 'IWM',
 'OXY',
 'PYPL']

In [64]:
database = r"TradingBotDatabase.db"
stock = get_stock_historic_data("TSLA", start = "2022-06-30", end = "2022-07-24")

# create a database connection
conn = create_connection(database)
stock.to_sql("stock", conn)

12197

In [9]:
database = r"TradingBotDatabase.db"

# create a database connection
conn = create_connection(database)

for symbol in collect_symbols:
    stock = get_stock_historic_data(symbol, start = "2022-07-22", end = "2022-08-02")
    stock.to_sql(symbol, conn, if_exists = 'append')

In [7]:
database = r"TradingBotDatabase.db"

# create a database connection
conn = create_connection(database)

stock = get_stock_historic_data("PYPL", start = "2022-07-22", end = "2022-08-02")
stock.to_sql("PYPL", conn, if_exists = 'append')

4649